In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
import os
os.chdir('/content/drive/My Drive')
!pwd

/content/drive/My Drive


In [7]:
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [8]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
# %matplotlib inline

import keras
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from keras.layers.advanced_activations import ReLU
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint

In [9]:
def load_dataset():
  train_data = pd.read_csv('trainset.csv')
  y_train = pd.read_csv('delta_theta.csv')
  target_data = pd.read_csv('target_data.csv')
  # target_label = pd.read_csv('target_label.csv')
  return train_data, y_train, target_data

In [10]:
# train_data.head()
# y_train.head()

In [11]:
train_data, y_train, target_data = load_dataset()

In [ ]:
# train_data.shape
# target_data.shape

In [12]:
num_classes = 4
train_data = np.array(train_data.iloc[:,:])
train_data = np.array([np.reshape(i,(128,128)) for i in train_data])
target_data = np.array(target_data.iloc[:,:])
target_data = np.array([np.reshape(i,(128,128)) for i in target_data])

In [13]:
train_data.shape
target_data.shape

(333, 128, 128)

In [14]:
train_data = train_data.astype('float32')
train_data = train_data/255

rgb_batch = np.repeat(train_data[..., np.newaxis], 3, -1)
rgb_batch.shape

target_data = target_data.astype('float32')
target_data = target_data/255

rgb_target = np.repeat(target_data[..., np.newaxis], 3, -1)
rgb_target.shape

(333, 128, 128, 3)

In [15]:
target_data = rgb_target

In [16]:
target_data.shape

(333, 128, 128, 3)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(rgb_batch, y_train, test_size = 0.3, random_state=666)
target_xtrain, target_xtest = train_test_split(target_data, test_size = 0.3, random_state=666)

In [18]:
input_shape = (128, 128, 3)

In [33]:
print(y_test)

     theta  R1  R2   B
3       55   7   0   2
184     15   7 -25  -3
302    -25  -8   0  12
239     -5  -3   0  -3
50      45  -3   0   2
..     ...  ..  ..  ..
14      55  -8   0   2
253     -5   7 -20   2
193      5 -18   0  12
54      45  -3   0  -3
177     15   7 -10   2

[100 rows x 4 columns]


In [35]:
def VGG_16(input_image):
    model = Conv2D(64, (3,3), padding="same", input_shape = input_shape)(input_image)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(64, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    # model = Dropout(0.25)(model)

    model = Conv2D(128, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(128, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    # model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)

    # model = Conv2D(256, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = Conv2D(256, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)


    return model

In [36]:
current_input = Input(shape=input_shape)
current_model = VGG_16(current_input)

target_input = Input(shape=input_shape)
target_model = VGG_16(target_input)

In [37]:
conv = concatenate([current_model, target_model])

In [38]:
conv = Flatten()(conv)
dense = Dense(512)(conv)
dense = ReLU(max_value=1.0)(dense)
dense = Dropout(0.25)(dense)
# dense = Dense(512)(dense)
# dense = ReLU(max_value=1.0)(dense)
# dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='tanh')(dense)

model = Model(inputs=[current_input, target_input], outputs=[output])

opt = SGD(learning_rate=0.01)

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])


In [39]:
best_weights_file="weights.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

In [41]:
model.fit([x_train, target_xtrain], y_train,
          epochs=15,
          validation_data=([x_test, target_xtest], y_test),
          callbacks=callbacks,
          verbose=1)

Epoch 1/15
8/8 [==============================] - 3s 329ms/step - loss: 289.1003 - accuracy: 0.5837 - val_loss: 273.2384 - val_accuracy: 0.5600

Epoch 00001: val_accuracy improved from -inf to 0.56000, saving model to weights.best.hdf5
Epoch 2/15
8/8 [==============================] - 2s 208ms/step - loss: 289.1882 - accuracy: 0.5837 - val_loss: 273.2382 - val_accuracy: 0.5600

Epoch 00002: val_accuracy did not improve from 0.56000
Epoch 3/15
8/8 [==============================] - 2s 208ms/step - loss: 289.1874 - accuracy: 0.5837 - val_loss: 273.2382 - val_accuracy: 0.5600

Epoch 00003: val_accuracy did not improve from 0.56000
Epoch 4/15
8/8 [==============================] - 2s 207ms/step - loss: 289.1877 - accuracy: 0.5837 - val_loss: 273.2380 - val_accuracy: 0.5600

Epoch 00004: val_accuracy did not improve from 0.56000
Epoch 5/15
8/8 [==============================] - 2s 209ms/step - loss: 289.1869 - accuracy: 0.5837 - val_loss: 273.2380 - val_accuracy: 0.5600

Epoch 00005: val_ac

In [42]:
model.evaluate([x_test, target_xtest], y_test)

4/4 [==============================] - 0s 50ms/step - loss: 273.2378 - accuracy: 0.5600


[273.2378234863281, 0.5600000023841858]

In [43]:
prediction = model.predict([x_test, target_xtest])
# prediction = np.argmax(prediction[10])

In [44]:
print(prediction)

[[ 0.9999995  -0.99996245 -0.9999805   0.99983114]
 [ 0.9999995  -0.9999653  -0.9999805   0.9998382 ]
 [ 0.9999996  -0.99995315 -0.99997866  0.99979836]
 [ 0.9999996  -0.99995977 -0.9999805   0.99981403]
 [ 0.9999996  -0.9999648  -0.9999805   0.999837  ]
 [ 0.9999996  -0.99996185 -0.99998075  0.99982804]
 [ 0.9999995  -0.9999652  -0.9999804   0.9998392 ]
 [ 0.9999994  -0.99995375 -0.999979    0.99980503]
 [ 0.9999996  -0.99996525 -0.99998057  0.99983895]
 [ 0.9999994  -0.99993974 -0.99997556  0.9997647 ]
 [ 0.9999996  -0.9999645  -0.99998057  0.9998365 ]
 [ 0.9999996  -0.99996275 -0.9999807   0.99983096]
 [ 0.9999995  -0.99996567 -0.9999802   0.999842  ]
 [ 0.9999995  -0.999963   -0.9999807   0.9998328 ]
 [ 0.9999996  -0.9999619  -0.99998075  0.9998275 ]
 [ 0.9999996  -0.9999644  -0.99998033  0.9998364 ]
 [ 0.9999996  -0.99995786 -0.999979    0.9998064 ]
 [ 0.99999946 -0.999946   -0.99997675  0.9997793 ]
 [ 0.99999946 -0.9999319  -0.9999739   0.999755  ]
 [ 0.9999996  -0.99996084 -0.99

In [ ]:
# import cv2
# from google.colab.patches import cv2_imshow
# img1 = cv2.imread('data65_27.png')
# img2 = cv2.imread('target_image_1.png')
#mg1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# img1 = cv2.resize(img1, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img1)
# img1.shape

# img1 = np.expand_dims(img1, axis = 0)
# img1 = np.expand_dims(img1, axis = 3)
# img1 = img1.astype('float32')
# img1 = img1/255
# plt.imshow(img1.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
# from PIL import Image
# # Open the image form working directory
# img1 = Image.open('/content/data65_27.png')
# img2 = Image.open('/content/target_image_1.png')

In [ ]:
# from matplotlib import image
# from matplotlib import pyplot
# # load image as pixel array
# img1 = image.imread('/content/data65_27.png')
# img2 = image.imread('/content/target_image_1.png')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('/content/data65_27.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img= cv2.resize(img, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img.shape

img = np.reshape(img, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img = img/255
img = np.repeat(img[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

error: ignored

In [ ]:
img.shape

In [ ]:
img2 = cv2.imread('/content/target_image_1.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2= cv2.resize(img2, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img2.shape

img2 = np.reshape(img2, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img2 = img2/255
img2 = np.repeat(img2[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img2.shape

In [ ]:
predictionimg = model.predict([img,img2])

In [ ]:
predictionimg